In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import calendar
import datetime
from multiprocessing.pool import ThreadPool

In [ ]:
index_url = "https://www.bseindia.com/indices/IndexArchiveData.html"
index = "S&P BSE 500"

In [ ]:
path = os.path.join(os.getcwd(),"Data")
print(path)

def download_index():

    # set from date
    def set_from_date(d,m,y):
        from_date = driver.find_element_by_xpath('//*[@id="txtFromDt"]')
        from_date.clear()
        from_date.click()
        year = driver.find_element_by_xpath('/html/body/div[4]/div/div/select[2]') 
        year = Select(year)
        while year.options[0].text > y:
            year.select_by_visible_text(year.options[0].text) 
            year = driver.find_element_by_xpath('/html/body/div[4]/div/div/select[2]') 
            year = Select(year)

        year.select_by_visible_text(y) 

        month = driver.find_element_by_xpath('/html/body/div[4]/div/div/select[1]') 
        month = Select(month)
        month.select_by_visible_text(m)  

        days=driver.find_element_by_xpath("//table/tbody/tr/td/a[text()="+str(d)+"]")
        days.click()

    # set to date
    def set_to_date(d,m,y):
        to_date = driver.find_element_by_xpath('//*[@id="txtToDt"]')
        to_date.clear()
        to_date.click()
        year = driver.find_element_by_xpath('/html/body/div[4]/div/div/select[2]') 
        year = Select(year)
        while year.options[0].text > y:
            print(year.options[0].text,y)
            year.select_by_visible_text(year.options[0].text) 
            year = driver.find_element_by_xpath('/html/body/div[4]/div/div/select[2]') 
            year = Select(year)

        year.select_by_visible_text(y) 

        month = driver.find_element_by_xpath('/html/body/div[4]/div/div/select[1]') 
        month = Select(month)
        month.select_by_visible_text(m)  

        days=driver.find_element_by_xpath("//table/tbody/tr/td/a[text()="+str(d)+"]")
        days.click()


    def set_index(index_):
        indexes = driver.find_element_by_xpath('//*[@id="ddlIndex"]')
        indexes = Select(indexes)
        indexes.select_by_visible_text(index_)  

    def download():
        submit = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div[5]/div/input')
        submit.click()
        time.sleep(1)
        driver.find_element_by_xpath("/html/body/div[2]/div/div[1]/div/div[1]/div[2]/i").click()
        time.sleep(3)
        driver.quit()

    def create_driver():
        chromeOptions = webdriver.ChromeOptions()
        chromeOptions.add_argument("--headless")
        chromeOptions.add_experimental_option("prefs",{"download.default_directory":path})
        driver = webdriver.Chrome(ChromeDriverManager().install(), options = chromeOptions)
        return driver

    if os.path.exists(os.path.join(os.getcwd(),"Data\\"+"Index.csv")):
        old_df = pd.read_csv(os.path.join(os.getcwd(),"Data\\"+"Index.csv"),names=["Date","Open","High","Low","Close"])
        old_df = old_df.iloc[1:]
        old_df["Date"] = pd.to_datetime(old_df["Date"])
        last = old_df["Date"].tail(1)[old_df.shape[0]]

        driver = create_driver()
        driver.get(index_url)
        set_index("S&P BSE 500")
        set_from_date(last.day,calendar.month_abbr[last.month],str(last.year))
        today = datetime.date.today()
        set_to_date(today.day,calendar.month_abbr[today.month],str(today.year))
        download()

        new_df = pd.read_csv(os.path.join(os.getcwd(),"Data\\"+"CSVForDate.csv"),names=["Date","Open","High","Low","Close"])
        new_df = new_df.iloc[1:]
        new_df["Date"] = pd.to_datetime(new_df["Date"])
        res = old_df.append(new_df)
        os.remove(os.path.join(os.getcwd(),"Data\\"+"Index.csv"))
        res.to_csv(os.path.join(os.getcwd(),"Data\\"+"Index.csv"),index=None)
        os.remove(os.path.join(os.getcwd(),"Data\\"+"CSVForDate.csv"))

    else:
        driver = create_driver()
        driver.get(index_url)
        set_index("S&P BSE 500")
        set_from_date("9","Aug","1999")
        today = datetime.date.today()
        set_to_date(today.day,calendar.month_abbr[today.month],str(today.year))
        download()
        os.rename(os.path.join(os.getcwd(),"Data\\"+"CSVForDate.csv"),os.path.join(path,"Index.csv"))


In [ ]:
download_index()